<a href="https://colab.research.google.com/github/RemyLpr/defi_ia/blob/main/feature_engineering/stagne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import des librairies et des données

In [1]:
import pandas as pd

In [21]:
df_vienna = pd.read_csv("vienna_danish_nono8_mobile0.csv")
df_vilnius = pd.read_csv("vilnius_danish_nono7_mobile0.csv")
df_amsterdam = pd.read_csv("amsterdam_belgian_nono3_mobile0.csv")
df_rome = pd.read_csv("rome_italian_nono6_mobile0.csv")
df_valletta = pd.read_csv("valletta_danish_nono9_mobile0.csv")
df_paris = pd.read_csv("paris.csv")
df_madrid = pd.read_csv("madrid.csv")
df_copenhagen = pd.read_csv("copenhagen.csv")
df_sofia = pd.read_csv("sofia.csv") # incomplet
df_list = [df_vienna, df_vilnius, df_amsterdam, df_rome, df_valletta, df_paris, df_madrid, df_copenhagen, df_sofia]

On doit renommer la colonne date dans les derniers dataframes pour uniformiser la dénomination de celle-ci.

In [26]:
df_paris.rename(columns = {'date':'request.date'}, inplace = True)
df_madrid.rename(columns = {'date':'request.date'}, inplace = True)
df_copenhagen.rename(columns = {'date':'request.date'}, inplace = True)
df_sofia.rename(columns = {'date':'request.date'}, inplace = True)

# Création de la feature stagne

Dans ce notebook on va, pour chaque ville, identifier quand stagne ou quand évolue le prix d'un hôtel. En effet, quelle que soit la requête, le prix d'un même hôtel augmente à des jours précis. Nous allons donc créer une nouvelle variable (binaire) indiquant pour le jour J si le prix à changé par rapport au jour J+1 (ce qui correspond au passé dans le référentiel de temps du défi). Pour effectuer ce travail on a besoin de données correspondant à la même requête pour chaque ville. Nous avons récupéré ces données sur la section data du Github.

In [32]:
df_stagn = pd.DataFrame()
for df in df_list:
  hotel_list = []
  is_stagnant_list = []
  for hotel in df.hotel_id.unique():
    df_hot = df[df["hotel_id"] == hotel]
    is_stagnant_list = []
    for date in range(0, 44): # on fait pas la première date
      df_hot_date_current = df_hot[df_hot["request.date"] == date]
      df_hot_date_previous = df_hot[df_hot["request.date"] == date+1]
      current_price = df_hot_date_current["price"].mean() # mean pour transformer le format juste, pas d'opération effectuée
      previous_price = df_hot_date_previous["price"].mean()
      is_stagnant_list.append(int(current_price == previous_price))
    # cas date = 45, par convention on dit que ça stagne pas
    is_stagnant_list.append(0)
    hotel_list.append(is_stagnant_list)
  df_stagn_city = pd.DataFrame(data = hotel_list)
  df_stagn_city["hotel_id"] = df.hotel_id.unique()
  df_stagn = pd.concat([df_stagn, df_stagn_city])

In [35]:
df_stagn.to_csv("stagne.csv", index = False)